In [1]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
neg_file = open("reviews.neg",encoding="utf-8")
pos_file = open("reviews.pos",encoding="utf-8")


with open("reviews.neg", encoding="utf-8") as neg_file:
    neg_content = neg_file.readlines()
neg_df = pd.DataFrame(neg_content, columns=['Review'])
neg_df['Label'] = 'Negative'


with open("reviews.pos", encoding="utf-8") as pos_file:
    pos_content = pos_file.readlines()
pos_df = pd.DataFrame(pos_content, columns=['Review'])
pos_df['Label'] = 'Positive'


merged_df = pd.concat([neg_df, pos_df], ignore_index=True)

df = merged_df.copy()


In [3]:
df.head()

,Review,Label
0,beklentimin altında bir ürün kaliteli değil\n,Negative
1,3. kademe hız da motor titreme yapıyor.bu sebe...,Negative
2,başlığı sabit durmuyor. arka koruma demiri üfl...,Negative
3,ürün gayet kalitesiz öğrenci evinde aldığımız ...,Negative
4,yuksek hizda kullanirken titredim yaptigi icin...,Negative


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235165 entries, 0 to 235164
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  235165 non-null  object
 1   Label   235165 non-null  object
dtypes: object(2)
memory usage: 3.6+ MB


In [5]:
df["Label"].value_counts()

Positive    220284
Negative     14881
Name: Label, dtype: int64

In [6]:
noktalama = string.punctuation
etkisiz = stopwords.words("Turkish")
print(noktalama)
print(etkisiz)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [7]:
for i in df["Review"].head():
    print(i + "\n-------------------")
    temp = ""
    for word in i.split():
        if word not in etkisiz and not word.isnumeric():
            temp += word + " "
    print(temp.rstrip() + "\n****************")      

beklentimin altında bir ürün kaliteli değil

-------------------
beklentimin altında bir ürün kaliteli değil
****************
3. kademe hız da motor titreme yapıyor.bu sebebten 3.kademe kullanımda uzun ömürlü olacağını zannetmiyorum.elide rahatsız ediyor.

-------------------
3. kademe hız motor titreme yapıyor.bu sebebten 3.kademe kullanımda uzun ömürlü olacağını zannetmiyorum.elide rahatsız ediyor.
****************
başlığı sabit durmuyor. arka koruma demiri üfleme hızını 2.sewiye yapınca yüksek ses cikariyor

-------------------
başlığı sabit durmuyor. arka koruma demiri üfleme hızını 2.sewiye yapınca yüksek ses cikariyor
****************
ürün gayet kalitesiz öğrenci evinde aldığımız 20 tl lil fön makinası ile aynı hiç farkı yok nerdeyse tek fark üzerinde arzum yazması o kadar.

-------------------
ürün gayet kalitesiz öğrenci evinde aldığımız tl lil fön makinası aynı farkı yok nerdeyse tek fark üzerinde arzum yazması kadar.
****************
yuksek hizda kullanirken titredim yaptigi 

In [8]:
for d in df["Review"].head():
    print(d + "\n-------------------")
    temp = ""
    for word in d.split():
        kelime = ''.join(char for char in word if char not in noktalama)
        if kelime:
            temp += kelime + " "
    print(temp.rstrip() + "\n****************")

beklentimin altında bir ürün kaliteli değil

-------------------
beklentimin altında bir ürün kaliteli değil
****************
3. kademe hız da motor titreme yapıyor.bu sebebten 3.kademe kullanımda uzun ömürlü olacağını zannetmiyorum.elide rahatsız ediyor.

-------------------
3 kademe hız da motor titreme yapıyorbu sebebten 3kademe kullanımda uzun ömürlü olacağını zannetmiyorumelide rahatsız ediyor
****************
başlığı sabit durmuyor. arka koruma demiri üfleme hızını 2.sewiye yapınca yüksek ses cikariyor

-------------------
başlığı sabit durmuyor arka koruma demiri üfleme hızını 2sewiye yapınca yüksek ses cikariyor
****************
ürün gayet kalitesiz öğrenci evinde aldığımız 20 tl lil fön makinası ile aynı hiç farkı yok nerdeyse tek fark üzerinde arzum yazması o kadar.

-------------------
ürün gayet kalitesiz öğrenci evinde aldığımız 20 tl lil fön makinası ile aynı hiç farkı yok nerdeyse tek fark üzerinde arzum yazması o kadar
****************
yuksek hizda kullanirken titredim 

In [9]:
df.to_csv(r'./cleaned.csv' , index = False)

In [10]:
df = pd.read_csv("cleaned.csv" )

In [11]:
df.head()

,Review,Label
0,beklentimin altında bir ürün kaliteli değil\n,Negative
1,3. kademe hız da motor titreme yapıyor.bu sebe...,Negative
2,başlığı sabit durmuyor. arka koruma demiri üfl...,Negative
3,ürün gayet kalitesiz öğrenci evinde aldığımız ...,Negative
4,yuksek hizda kullanirken titredim yaptigi icin...,Negative


In [12]:
X_train , X_test , y_train , y_test = train_test_split(df["Review"].values.astype('U'),
                                                      df["Label"].values.astype('U') , test_size = 0.2 , random_state = 42)

In [13]:
print(X_train.shape)

(188132,)


In [14]:
print(X_test.shape)

(47033,)


In [15]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

(188132, 186590)


In [16]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(188132, 186590)


In [17]:
clf = MultinomialNB().fit(X_train_tfidf , y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [18]:
y_pred = clf.predict(X_test_tfidf)
for Review , Sentiment in zip(X_test[:5] , y_pred[:]):
    print("%r => %s" % (Review , Sentiment))

'asus zenfone 2nin eksileri\n' => Positive
'performansı 9/10 tek özelliği iyon sisteminin olmaması fakat bunu sprey kullanarak cozebilirsiniz.te?ekkürler..\n' => Positive
'gayet başarılı kaliteli\n' => Positive
'önceden monster markalı laptop um gta 5 i 1 saat çalıştırırdı sonra işlemci ve anakart ısındığı için kapatırdı kendi kendine şimdi 4.5 saat dir gta oynuyorum pcde bir ısınma hisstemedim mükemmel bir ürün tavsiye ederim\n' => Positive
'bu fiyata çok güzel televizyon bence sorunsuz geldi\n' => Positive


In [19]:
print(accuracy_score(y_test , y_pred))

0.9375332213552187
